In [ ]:
pip install pdfplumber

In [ ]:
import pdfplumber as pdf
import pandas as pd
import re
from matplotlib import pyplot as plt

In [ ]:
#removing none
#removing ""
def table_to_df(pdf_file, month_int):
    month_pdf = pdf.open(pdf_file)
    pdf_page_3 = month_pdf.pages[3]
    #extract table in list format
    pdf_table = pdf_page_3.extract_table()
    edited_list = []
    #cleaning list to take out empty strings and None values
    for list_item in pdf_table:
        new_lista = []
        for list_itema in list_item:
            if list_itema != '':
                new_lista.append(list_itema)
        #can also add replace statement here to remove the commas
        new_listb = list(filter(None, new_lista))
        edited_list.append(new_listb)
    cropped_sales_list = edited_list[3:11]
    month_sales_df = pd.DataFrame(cropped_sales_list, columns = ["Product", "Quantity", "Total_Sales"])
    month_sales_df['Month'] = month_int
    return month_sales_df


In [ ]:
august_sales = table_to_df("August_2021.pdf", 8)

In [ ]:
july_sales = table_to_df("July_2021.pdf", 7)

In [ ]:
oct_sales = table_to_df("October_2021.pdf", 10)

In [ ]:
sept_sales = table_to_df("September_2021.pdf", 9)

In [ ]:
month_sales = [july_sales, august_sales, sept_sales, oct_sales]

In [ ]:
all_sales = pd.concat(month_sales)
#can make two dataframes, one containing sales data and the other with quantity data 

In [ ]:
only_sales = all_sales.drop(['Quantity'], axis=1)
only_sales

In [ ]:
sales_df = all_sales.pivot(index="Month", columns="Product", values="Total_Sales")
#this is so all column names are situated after pivoting
sales_df.reset_index(inplace=True)


In [ ]:
cols = ['Concentrate', 'Flower', 'Infused Liquid', 'Infused-Edible', 'Infused Non-Edible Solid', "Inhalable Compound Concentrate", "Kief", 'Shake/Trim', 'Vape Cartridge']
sales_df[cols] = sales_df[cols].replace({'\$':'', ',':''}, regex=True)
sales_df = sales_df.fillna(0)
sales_df

In [ ]:
sales_df.dtypes

In [ ]:
# changing columns from objects to floats to plot graphs
for col in sales_df.columns[1:]:
    sales_df = sales_df.astype({col:float})
    

In [ ]:
plt.plot(sales_df['Month'], sales_df['Concentrate'], label='Concentrate Sales')
plt.plot(sales_df['Month'], sales_df['Infused-Edible'], label='Edible Sales')
plt.plot(sales_df['Month'], sales_df['Flower'], label='Flower Sales')
plt.plot(sales_df['Month'], sales_df['Vape Cartridge'], label='Vape Sales')
plt.plot(sales_df['Month'], sales_df['Shake/Trim'], label='Shake and Trim Sales')
plt.xticks([7,8,9,10], ['July', 'August', 'September', 'October'])
plt.legend()
plt.ylim([500000,30000000])
plt.title('Total Sales per Product')
plt.xlabel('Months')
plt.ylabel('Sales($)')
plt.show()

In [ ]:
#plot multiple lines